In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from scipy.stats import randint
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
global df_traffic, resultados, modelo, modelo_clasificacion

In [2]:
df_traffic = pd.read_csv('Classifier.csv', sep=';')
df_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12283 entries, 0 to 12282
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   channelGrouping     12283 non-null  int64  
 1   date                12283 non-null  int64  
 2   fullVisitorId       12283 non-null  uint64 
 3   visitNumber         12283 non-null  int64  
 4   visitStartTime      12283 non-null  int64  
 5   browser             12283 non-null  int64  
 6   operatingSystem     12283 non-null  int64  
 7   isMobile            12283 non-null  bool   
 8   city                12283 non-null  int64  
 9   source              12283 non-null  int64  
 10  hits                12283 non-null  int64  
 11  pageviews           12283 non-null  int64  
 12  newVisits           12283 non-null  bool   
 13  transactionRevenue  12283 non-null  float64
 14  year                12283 non-null  int64  
 15  month               12283 non-null  int64  
 16  quar

In [3]:
# Dividir los datos en conjunto de entrenamiento y prueba
X = df_traffic.drop(columns=['transactionRevenue'])
y = df_traffic['transactionRevenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir los hiperparámetros del modelo
params = {
    'objective': 'reg:squarederror',  # La función de pérdida a optimizar
    'learning_rate': 0.1,  # Tasa de aprendizaje
    'max_depth': 3,  # Profundidad máxima del árbol
    'n_estimators': 100  # Número de árboles a entrenar
}

# Crear el modelo XGBoost
xgboost = xgb.XGBRegressor(**params)

# Entrenar el modelo
xgboost.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = xgboost.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 323.7505134377584


In [4]:
#definiendo el K - número de subconjuntos
xgb_reg = xgb.XGBRegressor()
cv = KFold(n_splits = 5, shuffle=True)

In [5]:
# Selección de hiperparámetros para la regresión lineal
param_dist = {
    'objective': ['reg:squarederror'],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'n_estimators': [50, 100, 150],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
}

In [6]:
# Configurar RandomizedSearchCV
randomized_search_xgb = RandomizedSearchCV(
    estimator=xgb_reg,
    param_distributions=param_dist,
    n_iter=20,
    scoring='neg_mean_squared_error',  
    cv=cv,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

In [7]:
# Ajustar el modelo
randomized_search_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=N...
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=None, ...),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.7, 0.8, 0.9],
                                        'learning_rate': [0.01, 0.1, 0.2],
                                        'max_depth': [3, 4, 5],
                                        'n_estimators': [50, 100, 150],
                                        'objective': ['reg:squarederror'],
                                        'subsample': [0.7, 0.8, 0.9]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [8]:
# Obtener los mejores parámetros y la mejor puntuación
best_params_rf = randomized_search_xgb.best_params_
best_score_rf = randomized_search_xgb.best_score_

print("Mejores parámetros:", best_params_rf)
print("Mejor puntuación:", best_score_rf)

Mejores parámetros: {'subsample': 0.9, 'objective': 'reg:squarederror', 'n_estimators': 150, 'max_depth': 3, 'learning_rate': 0.01, 'colsample_bytree': 0.9}
Mejor puntuación: -327.67825582896614


In [9]:
# Crear una nueva instancia de GradientBoostingRegressor con los mejores parámetros encontrados
best_xgb_model = xgb.XGBRegressor(**randomized_search_xgb.best_params_)

best_xgb_model.fit(X_train, y_train)

# Predecir con el modelo entrenado
y_pred = best_xgb_model.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calcular R^2
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

# Calcular RMSE
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

Mean Squared Error: 304.6819648108208
R^2 Score: 0.2904424203126379
Root Mean Squared Error: 17.455141500738996


In [10]:
model_scores = {
    'model': 'XGBoost',
    'mse': mse,
    'r2': r2,
    'rmse': rmse
}
print(model_scores)

{'model': 'XGBoost', 'mse': 304.6819648108208, 'r2': 0.2904424203126379, 'rmse': 17.455141500738996}


In [11]:
def save_scores(model_scores: dict):
  status = []
  #Utilizamos un try-except en caso de que el archivo no exista que cree uno
  try:
    #Si el archivo existe continúa por acá
    scores = pd.read_csv('scores.csv', sep=';')
    status.append('El archivo existe')
    exist = model_scores['model'] in scores['model'].values
    #Validamos si el modelo ya existe en el archivo de scores
    if (exist):
      #Si el modelo ya existe reemplazamos sus valores
      status.append('El modelo existe')
      criteria = scores['model'] == model_scores['model']
      index = scores[criteria].index[0]
      scores.iloc[index] = model_scores
      status.append('Se reemplazaron los valores del modelo')
    else:
      #Si el modelo no existe lo agregamos
      status.append('El modelo no existe')
      df_model_scores = pd.DataFrame(model_scores, index=[0])
      scores = pd.concat([scores, df_model_scores], ignore_index=True)
      status.append('Se añadió modelo nuevo y sus valores')
  except:
    #Si no existe el archivo lo creamos y cargamos los scores del modelo
    status.append('El archivo no existe')
    scores = pd.DataFrame(model_scores, index=[0])

  status.append('Se sobrescrive el archivo scores.csv con valores nuevos')
  scores.to_csv('scores.csv', sep=';', index=False)
  return status

In [12]:
save_scores(model_scores)

['El archivo existe',
 'El modelo no existe',
 'Se añadió modelo nuevo y sus valores',
 'Se sobrescrive el archivo scores.csv con valores nuevos']